# Jupyter notebooks to markdown
> using nbdev to convert notebooks to markdown

- date: 2020-11-10
- tags: [python]

This is very meta, but since I only update notebooks once in a while, this here notebook converts all the notebooks in the folder to markdown files and images, which are used to build the blog posts on the site.

The main reason to do this is to save time when rebuilding the blog on github pages, as installing nbdev and converting all the notebooks takes a while.

Later on could add an option to the config to either build notebooks on each build or to just read the converted markdown files.


In [7]:
#collapse-hide
import os
from pathlib import Path
from nbdev.export2html import convert_md

Settings used:

In [62]:
path_nb: Path = Path(".") # convert notebook is in the notebooks folder

path_md = path_nb / "markdown"
path_nb, path_md, str(path_md)

(Path('.'), Path('markdown'), 'markdown')

helper functions:

In [46]:
def make_folder(path, debug: bool = False):
    """makes parent folders if they don't exist for a path"""

    # assume the last thing is a file if there is a dot in the name
    if "." in path.name:
        path = path.parent

    try:
        path.mkdir(parents=True, exist_ok=False)
        if debug:
            print(f"{path} folder made")
    except FileExistsError:
        if debug:
            print(f"{path} folder already exists")
        pass

Now to convert all the jupyter notebooks to markdown files:

In [94]:
nb_paths = [
        f for f in path_nb.rglob("*.ipynb") if ".ipynb_checkpoints" not in str(f)
    ]

print(f"Converting {len(nb_paths)} notebooks to markdown files.")

for nb_path in nb_paths:
    print(f"converting {nb_path}")
    path_img = path_md / f"{nb_path.stem}_files"
    make_folder(path_img) # make _files folder inside markdown output folder
    convert_md(nb_path, dest_path=path_md, img_path=None, jekyll=True)
    # remove extra empty files folder made by nbdev
    Path(path_img.stem).rmdir()

Converting 9 notebooks to markdown files.
converting flood_fill.ipynb
converting knn.ipynb
converting blogging_with_jupyter_notebooks.ipynb
converting Advent-of-Code-2015.ipynb
converting monty_hall.ipynb
converting quicksort.ipynb
converting convert2md.ipynb
converting us_elections_2020.ipynb
converting monte_carlo_pi.ipynb


So now in the markdown output folder we have a list of `.md` files, with each file having a `_files` folder containing output artifacts for that notebook.

In [101]:
os.listdir(path_md)

['monte_carlo_pi_files',
 'monty_hall.md',
 'flood_fill_files',
 'us_elections_2020_files',
 'convert2md.md',
 'Advent-of-Code-2015_files',
 'Advent-of-Code-2015.md',
 'knn_files',
 'quicksort_files',
 'blogging_with_jupyter_notebooks.md',
 'blogging_with_jupyter_notebooks_files',
 'knn.md',
 'monte_carlo_pi.md',
 'convert2md_files',
 'flood_fill.md',
 'us_elections_2020.md',
 'quicksort.md',
 'monty_hall_files']

Taking a look at a sample md file:

In [107]:
txt = [p for p in path_md.rglob("*.md")][0].read_text()
print(txt[:100])

---
title: Monty Hall

date: 2019-07-05
tags: [python]
summary: "Monty Hall brain teaser"
description: "Monty Hall brain teaser"
---


The famous [Monty Hall brain teaser](https://en.wikipedia.org/wiki/Monty_Hall_problem):
> Suppose you're on a game show, and you're given the choice of three doors:Behind one door is a car; behind the others, goats. You pick a door, say No. 1, and the host, who knows what's behind the doors, opens another door, say No. 3, which has a goat. He then says to you, "Do you want to pick door No. 2?" Is it to your advantage to switch your choice?
There is a really [fun discussion over at Marilyn vos Savant's site](http://marilynvossavant.com/game-show-problem/).

Ok, now to setup the problem, along with some kind of visuals and what not.

## setting up a game

There are many ways to do this, but to keep it simple and human comprehensible I'm going to do it one game at a time. 

First up, a helper function which takes the door number guessed and the door opened